In [1]:
# Import necessary libraries
import os
from crewai import Agent, Task, Crew
from PyPDF2 import PdfReader
from crewai_tools import SerperDevTool, WebsiteSearchTool

# Configure API keys for external tools
os.environ["SERPER_API_KEY"] = ""
os.environ["OPENAI_API_KEY"] = ""


In [2]:
# Define the path to the PDF and the pages to extract
pdf_file_path = 'reports.pdf'
pages_to_extract = [1, 10]

def extract_text_from_pages(file_path, page_numbers):
    collected_text = ''
    with open(file_path, 'rb') as pdf_file:
        pdf_reader = PdfReader(pdf_file)
        for page_index in page_numbers:
            page = pdf_reader.pages[page_index - 1]
            text_content = page.extract_text()
            if text_content:
                collected_text += text_content
    return collected_text

# Extract text from the specified pages
extracted_text = extract_text_from_pages(pdf_file_path, pages_to_extract)
print(extracted_text)


Report Status    Male30 Years:
:
:
:Age
Gender
Reported        
P14/5/2023  11:03:00AMU439854467   DUMMY:
:
:
:
:Name        
Lab No.    
Ref By 
Collected       
A/c Status 16/5/2023  1:36:25PM
Final
Collected at            : Processed at             : LPL-ROHINI (NATIONAL REFERENCE LAB)
National Reference laboratory, Block E, Sector 
18, ROHINI
DELHI 110085LPL-NATIONAL REFERENCE LAB
National Reference laboratory, Block E, 
Sector 18, Rohini, New Delhi -110085
Test Report      
Test Name Results Units Bio. Ref. Interval
SwasthFit Super 4
COMPLETE BLOOD COUNT;CBC
Hemoglobin
(Photometry) 13.00 - 17.00 g/dL 15.00
Packed Cell Volume (PCV)
(Calculated) 40.00 - 50.00 % 45.00
RBC Count
(Electrical Impedence) 4.50 - 5.50 mill/mm3 4.50
MCV
(Electrical Impedence) 83.00 - 101.00 fL 90.00
MCH
(Calculated) 27.00 - 32.00 pg 32.00
MCHC
(Calculated) 31.50 - 34.50 g/dL 33.00
Red Cell Distribution Width (RDW)
(Electrical Impedence) 11.60 - 14.00 % 14.00
Total Leukocyte Count  (TLC)
(Electrical Impedenc

In [3]:
# Initialize tools for searching
serper_tool = SerperDevTool()
web_search_tool = WebsiteSearchTool()


In [4]:
# Create different agents with distinct roles and objectives
test_report_specialist = Agent(
    role='Test Report Specialist',
    goal='Examine the blood test report and provide a summary.',
    backstory='A specialist with expertise in analyzing blood test results.',
    verbose=True,
    allow_delegation=False
)

research_expert = Agent(
    role='Research Expert',
    goal='Find relevant health articles based on the test results.',
    backstory='A researcher skilled in locating health-related information.',
    tools=[serper_tool, web_search_tool],
    verbose=True,
    allow_delegation=False,
)

wellness_advisor = Agent(
    role='Wellness Advisor',
    goal='Offer health recommendations derived from the articles.',
    backstory='An advisor experienced in delivering health guidance.',
    verbose=True,
    allow_delegation=False,
)


In [5]:
# Define tasks to be handled by the agents
task_analyze_report = Task(
    description=f'Analyze the blood test report: "{extracted_text}"',
    expected_output='Summary of the blood test findings.',
    agent=test_report_specialist,
)

task_find_articles = Task(
    description='Look for health articles related to the blood test analysis.',
    expected_output='List of pertinent health articles with links.',
    agent=research_expert,
    context=[task_analyze_report]
)

task_provide_recommendations = Task(
    description='Generate health recommendations based on the identified articles.',
    expected_output='Health advice with links to relevant articles.',
    agent=wellness_advisor,
    context=[task_find_articles]
)


##  Use the API Key (Need to buy the API KEY form OPEN API then we can replace and the response will be generated)

In [6]:
# Assemble and initiate the crew
medical_crew = Crew(
    agents=[test_report_specialist, research_expert, wellness_advisor],
    tasks=[task_analyze_report, task_find_articles, task_provide_recommendations],
    verbose=2
)

# Begin task execution
medical_crew.kickoff()


 [2024-07-27 18:30:25][DEBUG]: == Working Agent: Test Report Specialist
 [2024-07-27 18:30:25][INFO]: == Starting Task: Analyze the blood test report: "Report Status    Male30 Years:
:
:
:Age
Gender
Reported        
P14/5/2023  11:03:00AMU439854467   DUMMY:
:
:
:
:Name        
Lab No.    
Ref By 
Collected       
A/c Status 16/5/2023  1:36:25PM
Final
Collected at            : Processed at             : LPL-ROHINI (NATIONAL REFERENCE LAB)
National Reference laboratory, Block E, Sector 
18, ROHINI
DELHI 110085LPL-NATIONAL REFERENCE LAB
National Reference laboratory, Block E, 
Sector 18, Rohini, New Delhi -110085
Test Report      
Test Name Results Units Bio. Ref. Interval
SwasthFit Super 4
COMPLETE BLOOD COUNT;CBC
Hemoglobin
(Photometry) 13.00 - 17.00 g/dL 15.00
Packed Cell Volume (PCV)
(Calculated) 40.00 - 50.00 % 45.00
RBC Count
(Electrical Impedence) 4.50 - 5.50 mill/mm3 4.50
MCV
(Electrical Impedence) 83.00 - 101.00 fL 90.00
MCH
(Calculated) 27.00 - 32.00 pg 32.00
MCHC
(Calculated) 3

NotFoundError: Error code: 404 - {'error': {'message': 'The model `gpt-4o` does not exist or you do not have access to it.', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}